# Mobilenetv2 Object detection using Custom dataset (In Progress)

**Note: the code is still in progress and is subjected to change**

The code below covers the following

- setting up the environment with all prerequisites packages that are needed to be installed for the API to work efficiently
- The preprocessing of the image dataset
- generation of tf.records
- generation of csv annotation file 
- Configuring the model with respect to the custom dataset
- Training of custom model 
- generating of inference graph of the trained model


## colab prerequisites and mounting it to drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


## **Change directory**

In [1]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


In [2]:
pwd

'/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection'

In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research


## check tensorflow version

In [7]:
! python -c 'import tensorflow as tf; print(tf.__version__)' 

2020-06-29 20:53:04.278510: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2.2.0


In [8]:
import tensorflow as tf
tf.test.is_gpu_available()
#tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## **Changing tensorflow version from 2.0.0 to 1.15.2**

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## Cloning the tensorflow repository

In [ ]:
! git clone https://github.com/osman-95/models.git


Cloning into 'models'...
remote: Enumerating objects: 35066, done.
remote: Total 35066 (delta 0), reused 0 (delta 0), pack-reused 35066
Receiving objects: 100% (35066/35066), 512.86 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (22765/22765), done.
Checking out files: 100% (2512/2512), done.


## **install prerequisites libraries**

In [4]:
! pip install --user Cython
! pip install --user contextlib2
! pip install --user pillow
! pip install --user lxml
! pip install --user jupyter
! pip install --user matplotlib

In [5]:
! pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

In [ ]:
#! protoc --python_out=. .\object_detection\protos\anchor_generator.proto 
! protoc --python_out=. ./object_detection/protos/argmax_matcher.proto  
protoc --python_out=. ./object_detection/protos/bipartite_matcher.proto 
protoc --python_out=. .\object_detection\protos\box_coder.proto 
protoc --python_out=. .\object_detection\protos\box_predictor.proto 
protoc --python_out=. .\object_detection\protos\eval.proto 
protoc --python_out=. .\object_detection\protos\faster_rcnn.proto 
protoc --python_out=. .\object_detection\protos\faster_rcnn_box_coder.proto 
protoc --python_out=. .\object_detection\protos\grid_anchor_generator.proto 
protoc --python_out=. .\object_detection\protos\hyperparams.proto 
protoc --python_out=. .\object_detection\protos\image_resizer.proto 
protoc --python_out=. .\object_detection\protos\input_reader.proto 
protoc --python_out=. .\object_detection\protos\losses.proto 
protoc --python_out=. .\object_detection\protos\matcher.proto 
protoc --python_out=. .\object_detection\protos\mean_stddev_box_coder.proto 
protoc --python_out=. .\object_detection\protos\model.proto 
protoc --python_out=. .\object_detection\protos\optimizer.proto 
protoc --python_out=. .\object_detection\protos\pipeline.proto 
protoc --python_out=. .\object_detection\protos\post_processing.proto 
protoc --python_out=. .\object_detection\protos\preprocessor.proto 
protoc --python_out=. .\object_detection\protos\region_similarity_calculator.proto 
protoc --python_out=. .\object_detection\protos\square_box_coder.proto 
protoc --python_out=. .\object_detection\protos\ssd.proto 
protoc --python_out=. .\object_detection\protos\ssd_anchor_generator.proto 
protoc --python_out=. .\object_detection\protos\string_int_label_map.proto 
protoc --python_out=. .\object_detection\protos\train.proto 
protoc --python_out=. .\object_detection\protos\keypoint_box_coder.proto 
protoc --python_out=. .\object_detection\protos\multiscale_anchor_generator.proto
protoc --python_out=. .\object_detection\protos\graph_rewriter.proto

## **Working Proto convertion code**
### **(converting the protos files to py)**

In [ ]:
! protoc --python_out=. ./object_detection/protos/*.proto  

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
! protoc --python_out=. ./object_detection/protos/anchor_generator.proto 

## **Converting xml files of image annotations to csv files**

In [ ]:
! python xml_to_csv.py

Successfully converted xml to csv.


In [4]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research


## **Building object detection prerequisites**

In [5]:
# From within TensorFlow/models/research/
! python setup.py build
! python setup.py install

running build
running build_py
running egg_info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
running install
running bdist_egg
running egg_info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux

In [6]:
import object_detection

In [7]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


## **TF records for train and test**

In [9]:
! python generate_tfrecord.py --csv_input=images/Train_annot_labels.csv  --image_dir=images/train --output_path=images/train.record



W0629 21:06:55.941634 139921927149440 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0629 21:06:56.106540 139921927149440 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images/train.record


In [ ]:

! python generate_tfrecord.py --csv_input=images/Test_annot_labels.csv  --image_dir=images/test --output_path=images/test.record



W0629 07:58:28.836865 140529563744128 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0629 07:58:28.914928 140529563744128 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images/test.record


In [10]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


## setting python path 

In [12]:

import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Project_tasks/Mobilenet/models/research/:/content/drive/My Drive/Project_tasks/Mobilenet/models/research/slim/'

## **Training of the object detection model with our custom images**

In [17]:
! python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v2_coco.config

Streaming output truncated to the last 5000 lines.
I0630 00:04:09.672671 139774429411200 learning.py:507] global step 17516: loss = 2.8909 (0.479 sec/step)
INFO:tensorflow:global step 17517: loss = 2.6692 (0.314 sec/step)
I0630 00:04:09.989294 139774429411200 learning.py:507] global step 17517: loss = 2.6692 (0.314 sec/step)
INFO:tensorflow:global step 17518: loss = 3.1173 (0.623 sec/step)
I0630 00:04:10.616515 139774429411200 learning.py:507] global step 17518: loss = 3.1173 (0.623 sec/step)
INFO:tensorflow:global step 17519: loss = 2.3535 (0.423 sec/step)
I0630 00:04:11.041227 139774429411200 learning.py:507] global step 17519: loss = 2.3535 (0.423 sec/step)
INFO:tensorflow:global step 17520: loss = 3.9700 (0.481 sec/step)
I0630 00:04:11.524027 139774429411200 learning.py:507] global step 17520: loss = 3.9700 (0.481 sec/step)
INFO:tensorflow:global step 17521: loss = 3.2374 (0.386 sec/step)
I0630 00:04:11.911579 139774429411200 learning.py:507] global step 17521: loss = 3.2374 (0.386

## **Exporting the inference graph**

In [18]:
! python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training/model.ckpt-20000 --output_directory inference_graph



W0630 00:33:10.116275 140187642636160 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0630 00:33:10.126518 140187642636160 module_wrapper.py:139] From /content/drive/My Drive/Project_tasks/Mobilenet/models/research/object_detection/exporter.py:419: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0630 00:33:10.153520 140187642636160 module_wrapper.py:139] From /content/drive/My Drive/Project_tasks/Mobilenet/models/research/object_detection/exporter.py:138: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0630 00:33:10.192601 140187642636160 module_wrapper.py:139] From /content/drive/My Drive/Project_tasks/Mobilenet/models/research/object_detection/core/preprocessor.py:3030: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0630 00:33:10.223766 140187642636160 module_wrapper.py:139] Fro